In [1]:
pip install feedparser requests beautifulsoup4 lxml pandas openpyxl

  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=b11a343d8af6248ed004fabf384036e549e4373d3087a99cea4e7592a953ecd8
  Stored in directory: /Users/hello_word/Library/Caches/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.


In [7]:
# -*- coding: utf-8 -*-
"""
Al Jazeera Arabic news scraper (RSS + fallback HTML scraping)
- يعرض الأخبار في الطرفية
- يصدّر إلى CSV و JSON (و XLSX اختياري)
"""

import re
import time
import json
import csv
import sys
from datetime import datetime
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# حاول تثبيت هذه الحزم عند الحاجة:
# pip install requests beautifulsoup4 lxml tqdm feedparser pandas openpyxl

try:
    import feedparser
except ImportError:
    feedparser = None

try:
    import pandas as pd
except ImportError:
    pd = None

BASE = "https://www.aljazeera.net"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/127.0.0.0 Safari/537.36"
}

# بعض RSS الشائعة (قد تتغير مع الوقت)
RSS_FEEDS = [
    # كل الأخبار (إن وجد)
    "https://www.aljazeera.net/rss",
    # السياسة
    "https://www.aljazeera.net/politics/rss",
    # اقتصاد
    "https://www.aljazeera.net/economy/rss",
    # رياضة
    "https://www.aljazeera.net/sports/rss",
    # تقارير
    "https://www.aljazeera.net/reports/rss",
]

# الحد الأقصى للأخبار التي نريدها
MAX_ITEMS = 100

def normalize_url(link: str) -> str:
    if not link:
        return ""
    if link.startswith("//"):
        return "https:" + link
    if link.startswith("/"):
        return urljoin(BASE, link)
    return link

def clean_text(x: str) -> str:
    if not x:
        return ""
    return re.sub(r"\s+", " ", x).strip()

def try_parse_date(val: str):
    for fmt in ("%a, %d %b %Y %H:%M:%S %z",
                "%Y-%m-%dT%H:%M:%S%z",
                "%Y-%m-%dT%H:%M:%S%zZ",
                "%Y-%m-%dT%H:%M:%S%zZ",
                "%Y-%m-%dT%H:%M:%S%z",
                "%Y-%m-%dT%H:%M:%S",
                "%Y-%m-%d %H:%M:%S",
                "%Y-%m-%d"):
        try:
            return datetime.strptime(val, fmt)
        except Exception:
            continue
    return None

def fetch(url):
    r = requests.get(url, headers=HEADERS, timeout=20)
    r.raise_for_status()
    return r

def parse_article_html(url: str) -> dict:
    """محاولة استخراج (العنوان، الملخص، التاريخ، القسم) من صفحة خبر فردية."""
    try:
        r = fetch(url)
    except Exception as e:
        return {"url": url, "error": str(e)}

    soup = BeautifulSoup(r.text, "lxml")

    # العنوان
    title = ""
    for sel in [
        'meta[property="og:title"]',
        "h1", 'meta[name="twitter:title"]'
    ]:
        node = soup.select_one(sel)
        if node:
            title = node.get("content") if node.has_attr("content") else node.get_text()
            title = clean_text(title)
            if title:
                break

    # الوصف
    summary = ""
    for sel in [
        'meta[name="description"]',
        'meta[property="og:description"]',
        'meta[name="twitter:description"]'
    ]:
        node = soup.select_one(sel)
        if node and node.get("content"):
            summary = clean_text(node["content"])
            if summary:
                break

    # التاريخ
    pub = ""
    for sel in [
        'meta[property="article:published_time"]',
        'meta[property="og:updated_time"]',
        'time[datetime]'
    ]:
        node = soup.select_one(sel)
        if node:
            pub = node.get("content") or node.get("datetime") or ""
            pub = clean_text(pub)
            if pub:
                break
    pub_dt = try_parse_date(pub) if pub else None
    pub_str = pub_dt.isoformat() if pub_dt else pub

    # القسم من الرابط أو فتات التنقل
    parsed = urlparse(url)
    category = ""
    parts = [p for p in parsed.path.split("/") if p]
    if parts:
        # مثال: /politics/2025/9/10/slug
        category = parts[0]

    return {
        "title": title or "",
        "summary": summary or "",
        "published": pub_str or "",
        "category": category or "",
        "url": url
    }

def scrape_homepage(max_links=50) -> list:
    """خطة احتياطية: نجمع روابط من الصفحة الرئيسية وبعض الصفحات الداخلية."""
    items = []
    seen = set()

    def collect_from(page_url):
        try:
            r = fetch(page_url)
        except Exception:
            return
        soup = BeautifulSoup(r.text, "lxml")
        # التقط كل الروابط الداخلية إلى مقالات
        for a in soup.find_all("a", href=True):
            href = normalize_url(a["href"])
            if not href.startswith(BASE):
                continue
            if any(x in href for x in ["video", "live", "programs"]):
                continue
            if re.search(r"/\d{4}/\d{1,2}/\d{1,2}/", href):  # يحتوي تاريخ في الرابط
                if href not in seen:
                    seen.add(href)
                    items.append(href)

    # ابدأ بالصفحة الرئيسية وبعض الأقسام المعروفة
    seeds = [
        BASE,
        f"{BASE}/politics",
        f"{BASE}/economy",
        f"{BASE}/sports",
        f"{BASE}/news",
        f"{BASE}/reports",
        f"{BASE}/opinions",
    ]
    for s in seeds:
        collect_from(s)
        time.sleep(1)
        if len(items) >= max_links:
            break

    items = items[:max_links]
    results = []
    for link in tqdm(items, desc="Parsing articles (HTML fallback)"):
        data = parse_article_html(link)
        results.append(data)
        time.sleep(0.8)  # لطفاً بالخادم
    return results

def read_via_rss(max_items=MAX_ITEMS) -> list:
    """محاولة قراءة الأخبار عبر RSS (الأفضل إن توفر)."""
    if feedparser is None:
        return []

    collected = []
    seen_links = set()
    for feed in RSS_FEEDS:
        try:
            d = feedparser.parse(feed)
        except Exception:
            continue
        for e in d.entries:
            link = normalize_url(getattr(e, "link", ""))
            if not link or link in seen_links:
                continue
            seen_links.add(link)

            title = clean_text(getattr(e, "title", ""))
            summary = clean_text(getattr(e, "summary", "")) or clean_text(getattr(e, "description", ""))
            published = ""
            if getattr(e, "published", ""):
                published = e.published
            elif getattr(e, "updated", ""):
                published = e.updated

            # جرّب تحويل التاريخ
            pub_dt = try_parse_date(published) if published else None
            pub_str = pub_dt.isoformat() if pub_dt else published

            # استنتاج القسم من الرابط
            parsed = urlparse(link)
            parts = [p for p in parsed.path.split("/") if p]
            category = parts[0] if parts else ""

            collected.append({
                "title": title,
                "summary": summary,
                "published": pub_str,
                "category": category,
                "url": link
            })
            if len(collected) >= max_items:
                break
        if len(collected) >= max_items:
            break

    return collected

def export_data(rows: list, basename="aljazeera_ar_news"):
    if not rows:
        print("لا توجد بيانات لتصديرها.")
        return

    # CSV
    csv_name = f"{basename}.csv"
    with open(csv_name, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(f, fieldnames=["title", "summary", "published", "category", "url"])
        w.writeheader()
        for r in rows:
            w.writerow(r)
    print(f"تم إنشاء الملف: {csv_name}")

    # JSON
    json_name = f"{basename}.json"
    with open(json_name, "w", encoding="utf-8") as f:
        json.dump(rows, f, ensure_ascii=False, indent=2)
    print(f"تم إنشاء الملف: {json_name}")

    # Excel (اختياري)
    if pd is not None:
        try:
            xlsx_name = f"{basename}.xlsx"
            df = pd.DataFrame(rows)
            df.to_excel(xlsx_name, index=False)
            print(f"تم إنشاء الملف: {xlsx_name}")
        except Exception as e:
            print("تعذر إنشاء Excel:", e)
    else:
        print("لإنشاء ملف Excel، ثبّت pandas و openpyxl.")

def main():
    print("🔎 المحاولة الأولى: RSS ...")
    news = read_via_rss(max_items=MAX_ITEMS)

    if not news:
        print("🔁 لم ينجح RSS، سنحاول Web Scraping الاحتياطي من الصفحات...")
        news = scrape_homepage(max_links=MAX_ITEMS)

    # إزالة الفراغات وترتيب حسب التاريخ إن أمكن
    for n in news:
        n["title"] = clean_text(n.get("title", ""))
        n["summary"] = clean_text(n.get("summary", ""))
        n["category"] = clean_text(n.get("category", ""))
        n["url"] = clean_text(n.get("url", ""))

    # طباعة أول 20 خبر للمعاينة
    print("\n=== أحدث الأخبار (أول 20) ===")
    for i, item in enumerate(news[:20], 1):
        print(f"\n[{i}] {item.get('title','')}")
        if item.get("published"):
            print(f"تاريخ: {item['published']}")
        if item.get("category"):
            print(f"قسم: {item['category']}")
        print(f"رابط: {item.get('url','')}")
        if item.get("summary"):
            print(f"ملخص: {item['summary'][:220]}{'...' if len(item['summary'])>220 else ''}")

    # تصدير
    export_data(news)

if __name__ == "__main__":
    # تذكير أخلاقي:
    # ✔️ راجع شروط استخدام الموقع و robots.txt
    # ✔️ استخدم تأخيراً بين الطلبات ولا تبالغ في عددها
    # ✔️ هذا الكود تعليمي؛ قد يلزم تعديله إذا تغيّر بناء صفحات الموقع
    try:
        main()
    except KeyboardInterrupt:
        print("\nتم الإيقاف بواسطة المستخدم.")
    except Exception as e:
        print("حدث خطأ:", e, file=sys.stderr)
        sys.exit(1)

🔎 المحاولة الأولى: RSS ...

=== أحدث الأخبار (أول 20) ===

[1] أوروبا تندد باختراق مسيّرات أجواء بولندا وروسيا تنفي مسؤوليتها
تاريخ: 2025-09-10T22:27:33+03:00
قسم: video
رابط: https://www.aljazeera.net/video/2025/9/11/%d8%a3%d9%88%d8%b1%d9%88%d8%a8%d8%a7-%d8%aa%d9%86%d8%af%d8%af-%d8%a8%d8%a7%d8%ae%d8%aa%d8%b1%d8%a7%d9%82-%d9%85%d8%b3%d9%8a%d8%b1%d8%a7%d8%aa-%d8%a3%d8%ac%d9%88%d8%a7%d8%a1?traffic_source=rss
ملخص: قال الأمين العام لحلف شمال الأطلسي (الناتو) مارك روته إن رد الحلف على انتهاك الطائرات المسيّرة الروسية للأجواء البولندية ناجح للغاية، مؤكدا أن الناتو جاهز للدفاع عن كل شبر من أراضيه.

[2] مقتل المؤثر الأميركي تشارلي كيرك وترامب يصفه بـ"الأسطوري العظيم"
تاريخ: 2025-09-10T22:22:03+03:00
قسم: news
رابط: https://www.aljazeera.net/news/2025/9/11/%d9%85%d9%82%d8%aa%d9%84-%d8%a7%d9%84%d9%85%d8%a4%d8%ab%d8%b1-%d8%a7%d9%84%d8%a3%d9%85%d9%8a%d8%b1%d9%83%d9%8a-%d8%aa%d8%b4%d8%a7%d8%b1%d9%84%d9%8a-%d9%83%d9%8a%d8%b1%d9%83?traffic_source=rss
ملخص: أعلن الرئيس الأميركي، مساء أمس الأربعاء، مق